In [ ]:
import os
import cv2
import numpy as np
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB

# Set the paths to your training and testing image folders
train_data_dir = '/kaggle/input/defungi-microscopic-fungi-image/train'
test_data_dir = '/kaggle/input/defungi-microscopic-fungi-image/test'

# Set the number of classes and the input size for the EfficientNet model
num_classes = 5
input_size =150

# Preprocess and augment the images
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Load and preprocess training images
train_images = []
train_labels = []

for class_name in os.listdir(train_data_dir):
    class_dir = os.path.join(train_data_dir, class_name)
    for image_name in os.listdir(class_dir):
        image_path = os.path.join(class_dir, image_name)
        image = cv2.imread(image_path)
        image = cv2.resize(image, (input_size, input_size))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        train_images.append(image)
        train_labels.append(class_name)

train_images = np.array(train_images)
train_labels = np.array(train_labels)

# Split the data into training and validation sets
train_images, val_images, train_labels, val_labels = train_test_split(train_images, train_labels, test_size=0.2, random_state=42)

# Load and preprocess testing images
test_images = []
test_labels = []

for class_name in os.listdir(test_data_dir):
    class_dir = os.path.join(test_data_dir, class_name)
    for image_name in os.listdir(class_dir):
        image_path = os.path.join(class_dir, image_name)
        image = cv2.imread(image_path)
        image = cv2.resize(image, (input_size, input_size))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        test_images.append(image)
        test_labels.append(class_name)

test_images = np.array(test_images)
test_labels = np.array(test_labels)

# Load the EfficientNet model with pre-trained weights
base_model = EfficientNetB3(weights='imagenet', include_top=False, input_shape=(input_size, input_size, 3))

# Extract features using EfficientNet model
train_features = base_model.predict(train_images)
val_features = base_model.predict(val_images)
test_features = base_model.predict(test_images)

# Flatten the features
train_features = train_features.reshape(train_features.shape[0], -1)
val_features = val_features.reshape(val_features.shape[0], -1)
test_features = test_features.reshape(test_features.shape[0], -1)

# Convert labels to strings
# train_labels = train_labels.astype(str)
# val_labels = val_labels.astype(str)
# test_labels = test_labels.astype(str)

# Convert labels to integers
label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_labels)
val_labels = label_encoder.transform(val_labels)
test_labels = label_encoder.transform(test_labels)

classifiers = [
    XGBClassifier(),
    AdaBoostClassifier(),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    SVC(),
    LogisticRegression(),
    GaussianNB()
]

for classifier in classifiers:
  classifier.fit(train_features, train_labels)
    # train_predictions = classifier.predict(train_features)
    # test_predictions = classifier.predict(test_features)
  print('Classifier:', classifier.__class__.__name__)
  classifier.fit(train_features, train_labels)

      # Make predictions on validation and testing set
  train_predictions = classifier.predict(train_features)
  val_predictions = classifier.predict(val_features)
  test_predictions = classifier.predict(test_features)

      # Calculate accuracy
  val_accuracy = accuracy_score(val_labels, val_predictions)
  test_accuracy = accuracy_score(test_labels, test_predictions)
      # Calculate training accuracy
  train_accuracy = accuracy_score(train_labels, train_predictions)

      # Print training accuracy
  print("Training Accuracy:", train_accuracy)
      # Print accuracy, classification report, and confusion matrix
  print("Validation Accuracy:", val_accuracy)
  print("Test Accuracy:", test_accuracy)
  print("Classification Report (Validation Set):")
  print(classification_report(val_labels, val_predictions))
  print("Classification Report (Test Set):")
  print(classification_report(test_labels, test_predictions))
      # Print classification report (training set)
  print("Classification Report (Training Set):")
  print(classification_report(train_labels, train_predictions))
  print("Confusion Matrix (Validation Set):")
  print(confusion_matrix(val_labels, val_predictions))
  print("Confusion Matrix (Test Set):")
  print(confusion_matrix(test_labels, test_predictions))


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


57/57 [==============================] - 4s 65ms/step
Classifier: XGBClassifier
Training Accuracy: 1.0
Validation Accuracy: 0.7333790267306374
Test Accuracy: 0.7218869994514536
Classification Report (Validation Set):
              precision    recall  f1-score   support

           0       0.67      0.44      0.53       401
           1       0.72      0.92      0.81       692
           2       0.88      0.74      0.80       108
           3       0.81      0.77      0.79       128
           4       0.77      0.58      0.66       130

    accuracy                           0.73      1459
   macro avg       0.77      0.69      0.72      1459
weighted avg       0.73      0.73      0.72      1459

Classification Report (Test Set):
              precision    recall  f1-score   support

           0       0.55      0.39      0.46       467
           1       0.73      0.91      0.81       881
           2       0.88      0.76      0.81       148
           3       0.81      0.72      0.76